In [ ]:
import csv,  os, fnmatch , glob,  sys , fnmatch ,  numpy as np , re , magic 
import  watchdog.events,  watchdog.observers, time , numpy as np, os.path 
import datatest as dt, pandas as pd, shutil, mmap , traceback  
from google.oauth2 import service_account
import google.auth
from google.cloud import bigquery_storage
import google.cloud.bigquery.magics
from google.cloud import bigquery
from configparser import ConfigParser
import logging,  logging.handlers, inspect,  ast
from logging.handlers import RotatingFileHandler
from logging import handlers

def Carga_Variable(seccion, campo) :
    try:
        cadena = parser.get(seccion, campo)
    except Exception as e: 
        msg('006', str(e))
        cadena='Vacio'
    finally:
        return cadena 
 
   
def Asigna_Variable() :
    global Lista_Tipos, File_Json , File_Lista, Dest_Rechazo, src_path, Dest_Salida,  Dest_Proceso, Dict_Mensaje
    global Prefijo_Rechazo, Prefijo_Proceso, Prefijo_Salida, Prefijo_Entrada, Abs_Entrada, Abs_Rechazo, Abs_Salida, Abs_Proceso 
    
    File_Lista      = os.path.join(Src_Base,Carga_Variable('paths', 'File_Lista'))
    Abs_Rechazo     = Carga_Variable('paths', 'Dest_Rechazo')
    Abs_Entrada     = Carga_Variable('paths', 'src_path')
    Abs_Salida      = Carga_Variable('paths', 'Dest_Salida')
    Abs_Proceso     = Carga_Variable('paths', 'Dest_Proceso')
    Dest_Rechazo    = os.path.join(Src_Base,Abs_Rechazo)
    src_path        = os.path.join(Src_Base,Abs_Entrada)
    Dest_Salida     = os.path.join(Src_Base,Abs_Salida) 
    Dest_Proceso    = os.path.join(Src_Base,Abs_Proceso)
    File_Json       = os.path.join(Src_Base,Carga_Variable('conexiones', 'Archivo_JSON'))
    Lista_Tipos     = ast.literal_eval(parser.get('formatos', 'Lista_Tipos'))
    Prefijo_Rechazo = Carga_Variable('prefijos', 'Rechazados')
    Prefijo_Proceso = Carga_Variable('prefijos', 'Procesados')
    Prefijo_Salida  = Carga_Variable('prefijos', 'Salida')
    
    seccion_log     = 'mensajes_log_info'
    if parser.has_section(seccion_log):
        Dict_Mensaje = dict(parser.items(seccion_log))
    return

class Pacc_Log(logging.Formatter):
    def format(self, record):
        record.message2 = record.args.get("message2")
        return super().format(record)

def Pacc_Logger():
    global logpac 
    try:
        logpac = logging.getLogger('pac')
        logpac.setLevel(logging.DEBUG)
        format = Pacc_Log('%(asctime)s - %(message2)s - %(name)s - %(levelname)s - %(message)s')
     
        conshan = logging.StreamHandler(sys.stdout)
        conshan.setLevel(logging.DEBUG)
        conshan.setFormatter(format)
        logpac.addHandler(conshan)

        filehan = handlers.RotatingFileHandler(os.path.join(os.getcwd(),'LOG', 'pacc.log'), maxBytes=(1048576*5), backupCount=7)
        filehan.setFormatter(format)
        logpac.addHandler(filehan)
        
    except Exception as error:
        print("Error with logs: %s" % (str(error)))    

def msg(codigo, texto): 
    logpac.info(texto, {"message2": Dict_Mensaje[codigo] })
    return
        
def Leer_Configuracion() : 
    global Src_Base, parser
    Src_Base = os.getcwd() 
    candidates = [os.path.join(Src_Base, 'PACC_Config.ini')]
    parser = ConfigParser()
    found = parser.read(candidates)
    missing = set(candidates) - set(found)
    if len(sorted(missing)) > 0 :
        pass
        msg('007', sorted(missing))
    else :
        try:
            Asigna_Variable()
        except Exception as e: 
            pass
            msg('008', sorted(missing))
            
def Credenciales(FileJson) :
    global bqclient , bqstorageclient 
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = FileJson
    credentials, your_project_id = google.auth.default(
        scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )
    bqclient = bigquery.Client(credentials=credentials, project=your_project_id,)
    bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)
    return

def Validar_Entorno():
    try:
        Leer_Configuracion()
        Pacc_Logger() 
        msg('010','')   
        Lista_Variables = [ File_Lista, Dest_Rechazo, src_path , Dest_Salida, Dest_Proceso  ,  File_Json ] 
        if len([f for f in Lista_Variables if not  os.path.exists(f) ]) > 0 : 
            msg('021', '') 
            return False 
        else :
            msg('009', '')
            try : 
                Credenciales(File_Json)
                msg('022', '')
                return True
            except Exception as e: 
                msg('023', str(e))
                return False
    except Exception as e: 
        return False
    
def rechazar(FileName, Dict_Matrix) :
    try:
        Destino_Rechazo = FileName.replace('.csv',Prefijo_Rechazo).replace(Abs_Entrada, Abs_Rechazo)
        Log_Rechazo = Destino_Rechazo.replace(Prefijo_Rechazo,'_LOG')
        new_path = shutil.move(FileName, Destino_Rechazo)
        with open(Log_Rechazo, 'w') as f:
            for item in Dict_Matrix:
                f.write("%s\n" % item)        
        
    except:
        print('041', new_path)
        #msg('041', new_path) 
    finally: 
        return 

def Procesar(FileName, Dict_Matrix) :
    try:
        Destino_Proceso = FileName.replace('.csv',Prefijo_Proceso).replace(Abs_Entrada,Abs_Proceso)
        Log_Proceso = Destino_Proceso.replace(Prefijo_Proceso,'_LOG.csv')
        new_path = shutil.move(FileName,Destino_Proceso)
        with open(Log_Proceso, 'w') as f:
            for item in Dict_Matrix:
                f.write("%s\n" % item)
        Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Procesar','Validacion':'SI' })
    except:
        print('026', FileName )
    finally: 
        return 
    
def Validar_Archivo(FileName) : 
    Dict_Matrix = [] 
    def Validar_Nombre() :
        Tipo_Proceso = FileName.split('_')[2] 
        nonlocal Dict_Matrix
        if Tipo_Proceso in Lista_Tipos:
            Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Nombre','Validacion':'SI' })
            msg('027', FileName )
        else :
            Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Nombre','Validacion':'NO' })
            msg('028', FileName )
        return Tipo_Proceso                        
    Tipo_Proceso = Validar_Nombre()
            
    def Validar_Preproceso() : 
        nonlocal Dict_Matrix
        with open(File_Lista, "a+") as f:
            f.seek(0, 0)
            if FileName in f.read():
                Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Preproceso','Validacion':'NO' })
                msg('029', FileName )
                return False                   
            else:
                Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Preproceso','Validacion':'SI' })  
                f.write("\n"+FileName)
                msg('030', FileName )
                return True                   
    Procesar = Validar_Preproceso()
        
    def Validar_Estructura() : 
        nonlocal Dict_Matrix
        df = pd.read_csv(FileName,index_col=False)
        if Tipo_Proceso == 'FONDO' :  
            try:
                    dt.validate(df.columns, {'TIPO_DOCUMENTO','NUMERO_DOCUMENTO','Periodo_resto_inicial'})
                    msg('035', FileName ) 
                    Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Estructura','Validacion':'SI' })
                    return True                    
            except:
                    Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Estructura','Validacion':'NO' })
                    msg('036', FileName )
                    return False   

        elif (Tipo_Proceso == 'PILA')   : 
            try:
                    dt.validate(df.columns, {'TIPO_DOCUMENTO','NUMERO_DOCUMENTO','Periodo_resto_inicial','Periodo_resto_final','Fecha_pago_inicial','Fecha_pago_final','INTERESES'})  
                    msg('035', FileName ) 
                    Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Estructura','Validacion':'SI' })
                    return True                   
            except:
                    msg('036', FileName )
                    Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Estructura','Validacion':'NO' })
                    return False                    
        else :
            Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Estructura','Validacion':'INDEFINIDA' })                           
            msg('037', FileName )
            return False 
    Estructura =  Validar_Estructura()                               
    
    def Validar_TipoCampos() : 
        nonlocal Dict_Matrix
        df = pd.read_csv(FileName,index_col=False)
        dt.register_accessors()
        if Tipo_Proceso == 'FONDO' :  
            try: 
                df = df.astype(dtype={'Periodo_resto_inicial': 'string' })
                df[['TIPO_DOCUMENTO','NUMERO_DOCUMENTO','Periodo_resto_inicial']].validate((str,int,str))
                Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Campos','Validacion':'SI' })                           
                msg('038', FileName )
                return True 
            except Exception as e: 
                Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Campos','Validacion':'NO' })
                msg('039', FileName )
                return False
        if Tipo_Proceso == 'PILA' :
            try: 
                df = df.astype(dtype={'Periodo_resto_inicial': 'string', 'Periodo_resto_final': 'string','Fecha_pago_inicial': 'string','Fecha_pago_final' : 'string' })
                df[['TIPO_DOCUMENTO','NUMERO_DOCUMENTO','Periodo_resto_inicial','Periodo_resto_final','Fecha_pago_inicial','Fecha_pago_final','INTERESES']].validate((str,int,str,str,str,str,int))
                Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Campos','Validacion':'SI' })                           
                msg('038', FileName )
                return True 
            except Exception as e: 
                Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Campos','Validacion':'NO' })
                msg('039', FileName )
                return False
    campos = Validar_TipoCampos() 
   
    if ( Tipo_Proceso in Lista_Tipos and Procesar == True  and Estructura == True and campos == True ) : 
        return Tipo_Proceso , Dict_Matrix  
    else :  
        Tipo_Proceso = "INVALIDO"
        msg('020',FileName)
        rechazar(FileName, Dict_Matrix)
        return Tipo_Proceso, Dict_Matrix 
    
def Carga_Bq(Tipo_Proceso, FileName) : 
    df = pd.read_csv(FileName,index_col=False)
    if (Tipo_Proceso == 'FONDO') : 
        df = df.astype(dtype={'Periodo_resto_inicial': 'string' })
        table_id = "co-ugpp-datalake-prd.piloto_data.FONDO_raw"
        job_config = bigquery.LoadJobConfig(
            schema=[],
            write_disposition="WRITE_TRUNCATE",)
        
    elif (Tipo_Proceso =='PILA') :
        df = df.astype(dtype={'Periodo_resto_inicial': 'string', 'Periodo_resto_final': 'string','Fecha_pago_inicial': 'string','Fecha_pago_final' : 'string' })
        table_id = "co-ugpp-datalake-prd.piloto_data.PILA_raw"
        job_config = bigquery.LoadJobConfig(
            schema=[],
            write_disposition="WRITE_TRUNCATE",)
    try:
        job = bqclient.load_table_from_dataframe( df, table_id, job_config=job_config )  
        job.result()  
        table = bqclient.get_table(table_id)   
        print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id ) ) 
        return  
    except Exception as e: 
        msg('50', str(e)) 
        return  

def Consulta_Bq(Tipo_Proceso, FileName) : 
    if (Tipo_Proceso == 'FONDO') : 
        query_string = """ CALL piloto_data.sp_proc_fondo()  """
    elif (Tipo_Proceso == 'PILA') :
        query_string = """ CALL piloto_data.sp_proc_pila() """
    try:
        dataframe = (bqclient.query(query_string).result().to_dataframe(bqstorage_client=bqstorageclient))
        NombreSalida = FileName.replace('.csv',Prefijo_Salida).replace(Abs_Entrada, Abs_Salida)
        dataframe.to_csv(NombreSalida, sep=',', encoding='utf-8', index=False)
        print(dataframe, dataframe.dtypes )
        return True
    except Exception as e: 
        msg('51', str(e))

def Limpiar_Lista(FileName) :
    with open(File_Lista, 'r') as file :
        lines = file.readlines()
    with open(File_Lista, 'w') as file :  
        for line in lines:
            if line.strip("\n") != FileName:
                file.write(line)   
    return           
        
def Cerrar_Proceso(Tipo_Proceso, Dict_Matrix, FileName) : 
       
    if Tipo_Proceso in Lista_Tipos:
        try:
            Carga_Bq(Tipo_Proceso, FileName )
            Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Cargue_BQ','Validacion':'SI' })
        except Exception as e: 
            Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Cargue_BQ','Validacion':'NO' })                           
            Limpiar_Lista(FileName)
        try:
            Consulta_Bq(Tipo_Proceso, FileName )
            Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Consulta_BQ','Validacion':'SI' })
            Procesar(FileName, Dict_Matrix) 
        except Exception as e: 
            Dict_Matrix.append({'Archivo': FileName, 'Tipo_Proceso': Tipo_Proceso, 'Funcion':'Consulta_BQ','Validacion':'NO' })                           
            Limpiar_Lista(FileName) 
    return 

class Handler(watchdog.events.PatternMatchingEventHandler):
    def __init__(self):
        watchdog.events.PatternMatchingEventHandler.__init__(self, patterns=['*.csv'], ignore_directories=True, case_sensitive=False)

    def on_created(self, event):
        msg('033','')
        FileName=event.src_path 
        Tipo_Proceso, Dict_Matrix = Validar_Archivo(FileName)
        Cerrar_Proceso(Tipo_Proceso, Dict_Matrix, FileName)
               
if __name__ == "__main__":
    try:
        if Validar_Entorno() : 
            msg('011','')
            event_handler = Handler()
            observer = watchdog.observers.Observer()
            observer.schedule(event_handler, path=src_path, recursive=True)
            observer.start()
            try:
                while True:
                    time.sleep(20)
            except KeyboardInterrupt:
                observer.join()
        else: 
            pass
        msg('000','')
    except: 
            msg('999','')
            sys.exit()